** The softmax algorithm ** 
- We need to make our bandit algorithm care about the known differences betweeen the estimated values of the arms when our algorithm decides decides which arm to explore.
- We need _**structured exploration**_ rather than the _**haphazard exploration**_ that the epsilon-Greedy algorithm provides.
- The softmax algorithm tries to cope up with arms differing in estimated value by explicitly incorporating information about the reward rates of the available arms into its method of choosing which arm to select when it explores.


In [20]:
import math
import random
import numpy

import nbimporter
import epsilonGreedy
import DebuggingBanditAlgorithms

In [14]:
def categorical_draw(probs):
    z = random.random()
    cum_prob = 0.0
    for idx, prob in enumerate( probs ):
        cum_prob += prob
        if cum_prob > z :
            return idx        
    return idx 

In [15]:


class SoftMax():
    
    '''
        This class implements Softmax Multiarm Bandit algorithm
    '''

    def __init__( self, temperature, counts=[], values=[] ):
        '''
            temperature : Systems behave randomly at high temperature, while they take on more structure at low temperature
           
            counts  : A vector of integers of length N that tells us how many time we have played 
            each of the N arms available to us in the current bandit problem.
            
            values  : A vector of floating point numbers that defines the average amount of reward we have gotten 
            when playing each of the N arms available to us.
        '''
        self.temperature = temperature
        self.counts = counts
        self.values = values
        
    def __repr__(self):
        return 'EpsilonGreedy({:.2f},{!r}, {!r})'.format(self.temperature, self.counts, self.values)
    
    def initialize(self, n_arms):
        # Intializing / reset rewards & counts to zeros for each arm(or option)
        self.counts = [ 0 for col in range(n_arms) ]
        self.values = [ 0.0 for col in range(n_arms) ]
    
    def select_arm(self):
        ''' Returns the index of the next arm to pull '''
        values = [ numpy.exp( v / self.temperature ) for v in self.values ]
        z = sum( values )
        probs = [ v / z for v in values ]
        return categorical_draw(probs)
    
    def update(self, chosen_arm, reward):
        '''        
        After we pull an arm, we get a reward signal back from our system. This function update our algorithm's beliefs
        about the quality of the arm we just chose by providing this reward information.
        
        chosen_arm : The numeric index of the most recently chosen arm
        reward     : The reward received from chossing that arm
        '''
        self.counts[ chosen_arm ] += 1
        n = self.counts[ chosen_arm ]
        value = self.values[chosen_arm]
        new_value = ( ( n-1 ) * value + reward ) / float(n) 
        self.values[chosen_arm] = new_value
    


In [6]:
from epsilonGreedy import EpsilonGreedy
from DebuggingBanditAlgorithms import BernoulliArm
from DebuggingBanditAlgorithms import testing_algorithm
random.seed(1)
means = [ 0.1, 0.1, 0.1, 0.1, 0.9]
n_arms = len(means)
random.shuffle(means)
arms = [  BernoulliArm(mu) for mu in means ]

f = open( 'standard_softmax_results.tsv', 'w')
for temperature in [ 0.1, 0.2, 0.3, 0.4, 0.5] :
    algo = SoftMax ( temperature )
    algo.initialize(n_arms)
    results = testing_algorithm ( algo, arms, 5000, 250 )
    for i in range( len(results[0]) ):
        f.write( str(temperature)  + '\t')
        f.write( '\t'.join( [ str( results[j][i]) for j in range( len(results) ) ] )  + '\n')
f.close()

#### The Annealing Softmax Algorithm
- Annealing is the process of decreasing the temperature in a Softmax algorithm over the time.
- Annealing is the process of modifying a bandit algorithm's behavior so that it will explore less over time

In [16]:


class AnnealingSoftMax():
    
    '''
        This class implements Softmax Multiarm Bandit algorithm
    '''

    def __init__( self,  counts=[], values=[] ):
        '''
            temperature : Systems behave randomly at high temperature, while they take on more structure at low temperature
           
            counts  : A vector of integers of length N that tells us how many time we have played 
            each of the N arms available to us in the current bandit problem.
            
            values  : A vector of floating point numbers that defines the average amount of reward we have gotten 
            when playing each of the N arms available to us.
        '''
        # self.temperature = temperature
        self.counts = counts
        self.values = values
        
    def __repr__(self):
        return 'EpsilonGreedy({!r}, {!r})'.format( self.counts, self.values)
    
    def initialize(self, n_arms):
        # Intializing / reset rewards & counts to zeros for each arm(or option)
        self.counts = [ 0 for col in range(n_arms) ]
        self.values = [ 0.0 for col in range(n_arms) ]
    
    def select_arm(self):
        ''' Returns the index of the next arm to pull '''
        t = sum(self.counts) + 1
        temperature = 1 / math.log(t + 0.0000001 )
        values = [ numpy.exp( v / temperature ) for v in self.values ]
        z = sum( values )
        probs = [ v / z for v in values ]
        return categorical_draw(probs)
    
    def update(self, chosen_arm, reward):
        '''        
        After we pull an arm, we get a reward signal back from our system. This function update our algorithm's beliefs
        about the quality of the arm we just chose by providing this reward information.
        
        chosen_arm : The numeric index of the most recently chosen arm
        reward     : The reward received from chossing that arm
        '''
        self.counts[ chosen_arm ] += 1
        n = self.counts[ chosen_arm ]
        value = self.values[chosen_arm]
        new_value = ( ( n-1 ) * value + reward ) / float(n) 
        self.values[chosen_arm] = new_value
    

    

In [3]:
import numpy
import random
import math

import nbimporter
import epsilonGreedy
import DebuggingBanditAlgorithms

from epsilonGreedy import EpsilonGreedy
from DebuggingBanditAlgorithms import BernoulliArm
from DebuggingBanditAlgorithms import testing_algorithm
random.seed(1)
means = [ 0.1, 0.1, 0.1, 0.1, 0.9]
n_arms = len(means)
random.shuffle(means)
arms = [  BernoulliArm(mu) for mu in means ]

f = open( 'annealing_softmax_results.tsv', 'w')

algo = AnnealingSoftMax ( )
algo.initialize(n_arms)
results = testing_algorithm ( algo, arms, 5000, 250 )
for i in range( len(results[0]) ):
    f.write( '\t'.join( [ str( results[j][i]) for j in range( len(results) ) ] )  + '\n')
f.close()